# Geospatial Data Analysis Lab: Steel Plants Dataset


**(15/10/2025) Learning Objectives:**
- Perform exploratory data analysis (EDA) on geospatial datasets
- Visualize geospatial data using interactive maps with Plotly
- Merge environmental data with asset locations
- Aggregate data at the company level
- Integrate geospatial visualizations into a Streamlit dashboard

---


## Part 1: Setup and Data Loading

Import the necessary libraries and load the steel plants dataset.


In [11]:
# Import required libraries
# - pandas for data manipulation
# - numpy for numerical operations
# - plotly.express and plotly.graph_objects for interactive visualizations
# - Any other libraries you might need

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go



In [12]:
# Load the steel plants dataset from Excel file
# The dataset has multiple sheets - we need to merge "Plant data" and "Plant capacities and status"
excel_file = 'Dataset/Plant-level-data-Global-Iron-and-Steel-Tracker-September-2025-V1.xlsx'

# Load both sheets
plant_data = pd.read_excel(excel_file, sheet_name='Plant data')
capacity_data = pd.read_excel(excel_file, sheet_name='Plant capacities and status')

print(f"Plant data shape: {plant_data.shape}")
print(f"Capacity data shape: {capacity_data.shape}")

# Merge on Plant ID - using left merge to keep all plants from plant_data
# Some plants may have multiple capacity records, so we'll need to handle duplicates
df = pd.merge(plant_data, capacity_data, on='Plant ID', how='left', suffixes=('', '_capacity'))

print(f"\nMerged data shape (before filtering): {df.shape}")

# Filter to keep only operating plants
print(f"\nStatus distribution before filtering:")
print(df['Status'].value_counts())

df = df[df['Status'] == 'operating'].copy()

print(f"\nAfter filtering for 'operating' status:")
print(f"Final dataset shape: {df.shape}")
print(f"Number of operating plants: {len(df)}")

# Display first few rows
df.head()

Plant data shape: (1209, 44)
Capacity data shape: (1744, 15)

Merged data shape (before filtering): (1744, 58)

Status distribution before filtering:
Status
operating                    868
announced                    285
retired                      181
construction                 151
operating pre-retirement     125
mothballed                    73
cancelled                     59
mothballed pre-retirement      2
Name: count, dtype: int64

After filtering for 'operating' status:
Final dataset shape: (868, 58)
Number of operating plants: 868


,Plant ID,Plant name (English),Plant name (other language),Other plant names (English),Other plant names (other language),Owner,Owner (other language),Owner GEM ID,Owner PermID,SOE Status,...,Start date_capacity,Nominal crude steel capacity (ttpa),Nominal BOF steel capacity (ttpa),Nominal EAF steel capacity (ttpa),Nominal OHF steel capacity (ttpa),Other/unspecified steel capacity (ttpa),Nominal iron capacity (ttpa),Nominal BF capacity (ttpa),Nominal DRI capacity (ttpa),Other/unspecified iron capacity (ttpa)
1,P100000120439,Algerian Qatari Steel Jijel plant,الجزائرية القطرية للصلب,AQS,NaN,Algerian Qatari Steel,NaN,E100001000957,5076384326,Partial,...,2017,2200,NaN,2200,NaN,NaN,2500,NaN,2500,NaN
4,P100000121198,Ozmert Algeria steel plant,NaN,NaN,NaN,Ozmert Algeria SARL,NaN,E100001012196,unknown,NaN,...,unknown,800,NaN,800,NaN,NaN,500,NaN,500,NaN
5,P100000120440,Sider El Hadjar Annaba steel plant,مركب الحجار للحديد والصلب,"ArcelorMittal Annaba (predecessor), El Hadjar ...",NaN,Groupe Industriel Sider SpA,NaN,E100001000960,5000941519,Full,...,1969,2150,350.0,1800,NaN,NaN,1500,1500,NaN,NaN
7,P100000120441,Tosyali Algerie Oran steel plant,شركة توسيالي الجزائرية التركية للحديد والصلب,NaN,Tosyali Algérie,Tosyali Ironsteel Industry Algerie SpA,NaN,E100000131071,5074196906,NaN,...,2013,6200,NaN,6200,NaN,NaN,5000,NaN,5000,NaN
8,P100000120005,Aceria Angola Bengo steel plant,NaN,ADA Steel,NaN,Ada - Aceria De Angola SA,NaN,E100000131097,unknown,NaN,...,2015,500,NaN,NaN,NaN,500,NaN,NaN,NaN,NaN


---
## Part 2: Exploratory Data Analysis (15 minutes)

Answer the following questions through your analysis:


### Question 1: Data Overview
**Task:** Display basic information about the dataset.
- How many steel plants are in the dataset?
- What are the column names and data types?
- Are there any missing values?


In [13]:
# Display dataset shape
print("Number of steel plants:", df.shape[0])
print("Dataset shape (rows, columns):", df.shape)

Number of steel plants: 868
Dataset shape (rows, columns): (868, 58)


In [14]:
# Display column information and data types
print("\nColumn names and data types:")
print(df.info())


Column names and data types:
<class 'pandas.core.frame.DataFrame'>
Index: 868 entries, 1 to 1742
Data columns (total 58 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Plant ID                                 868 non-null    object 
 1   Plant name (English)                     868 non-null    object 
 2   Plant name (other language)              533 non-null    object 
 3   Other plant names (English)              549 non-null    object 
 4   Other plant names (other language)       222 non-null    object 
 5   Owner                                    868 non-null    object 
 6   Owner (other language)                   411 non-null    object 
 7   Owner GEM ID                             868 non-null    object 
 8   Owner PermID                             868 non-null    object 
 9   SOE Status                               148 non-null    object 
 10  Parent                  

In [15]:
# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())
total_missing = df.isnull().sum().sum()
print("\nTotal number of missing values in dataset:", total_missing)


Missing values per column:
Plant ID                                     0
Plant name (English)                         0
Plant name (other language)                335
Other plant names (English)                319
Other plant names (other language)         646
Owner                                        0
Owner (other language)                     457
Owner GEM ID                                 0
Owner PermID                                 0
SOE Status                                 720
Parent                                       0
Parent GEM ID                                0
Parent PermID                                0
Location address                             0
Municipality                                 0
Subnational unit (province/state)            0
Country/Area                                 0
Region                                       0
Other language location address            545
Coordinates                                  0
Coordinate accuracy             

### Question 2: Statistical Summary
**Task:** Generate descriptive statistics for numerical columns.
- What is the average plant capacity?
- What is the range of latitudes and longitudes?
- What is the distribution of plant ages?


What is the average plant capacity?

In [ ]:
capacity_column = 'Nominal crude steel capacity (ttpa)'

# Convert the selected capacity column to numeric
if capacity_column in df.columns:
    df[capacity_column] = pd.to_numeric(df[capacity_column], errors='coerce')

print("\nCapacity Statistics (in thousand tonnes per annum - ttpa):\n")

if capacity_column in df.columns:
    non_null_count = df[capacity_column].notna().sum()
    if non_null_count > 0:
        print(f"{capacity_column}:")
        print(f"  Plants with data: {non_null_count}")
        print(f"  Average: {df[capacity_column].mean():,.2f} ttpa")
        print(f"  Median: {df[capacity_column].median():,.2f} ttpa")
        print(f"  Min: {df[capacity_column].min():,.2f} ttpa")
        print(f"  Max: {df[capacity_column].max():,.2f} ttpa")
        print(f"  Total: {df[capacity_column].sum():,.2f} ttpa")
        print()


# 1. Histogram of crude steel capacity distribution
fig1 = px.histogram(
    df.dropna(subset=['Nominal crude steel capacity (ttpa)']),
    x='Nominal crude steel capacity (ttpa)',
    nbins=50,
    title='Distribution of Crude Steel Capacity (Operating Plants)',
    labels={'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)'},
    color_discrete_sequence=['#1f77b4']
)
fig1.update_layout(
    xaxis_title='Capacity (thousand tonnes per annum)',
    yaxis_title='Number of Plants',
    showlegend=False,
    height=400
)
fig1.show()

# 2. Top 20 plants by crude steel capacity
top_plants = df.nlargest(20, 'Nominal crude steel capacity (ttpa)')[
    ['Plant name (English)', 'Country/Area', 'Nominal crude steel capacity (ttpa)', 'Owner']
].copy()

fig2 = px.bar(
    top_plants,
    y='Plant name (English)',
    x='Nominal crude steel capacity (ttpa)',
    title='Top 20 Operating Plants by Crude Steel Capacity',
    labels={'Nominal crude steel capacity (ttpa)': 'Capacity (ttpa)', 'Plant name (English)': 'Plant'},
    hover_data=['Country/Area', 'Owner'],
    color='Nominal crude steel capacity (ttpa)',
    color_continuous_scale='Viridis',
    orientation='h'
)
fig2.update_layout(
    xaxis_title='Capacity (ttpa)',
    yaxis_title='',
    height=600,
    showlegend=False
)
fig2.show()


Capacity Statistics (in thousand tonnes per annum - ttpa):

Nominal crude steel capacity (ttpa):
  Plants with data: 815
  Average: 2,459.65 ttpa
  Median: 1,350.00 ttpa
  Min: 13.00 ttpa
  Max: 22,999.00 ttpa
  Total: 2,004,616.52 ttpa



What is the range of latitudes and longitudes?

In [19]:
# Split coordinates column into latitude and longitude
df[['latitude', 'longitude']] = df['Coordinates'].str.split(',', expand=True)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

print(f"Latitude range:")
print(f"  Min: {df['latitude'].min():.4f}°")
print(f"  Max: {df['latitude'].max():.4f}°")
print(f"  Mean: {df['latitude'].mean():.4f}°")
print(f"\nLongitude range:")
print(f"  Min: {df['longitude'].min():.4f}°")
print(f"  Max: {df['longitude'].max():.4f}°")
print(f"  Mean: {df['longitude'].mean():.4f}°")

# Filter out plants without capacity data for better visualization
df_with_capacity = df.dropna(subset=['Nominal crude steel capacity (ttpa)'])

print(f"\nPlants with capacity data for visualization: {len(df_with_capacity)} out of {len(df)}")

# Create scatter geo map showing operating plants with capacity data
fig_geo = px.scatter_geo(
    df_with_capacity,
    lat='latitude',
    lon='longitude',
    hover_name='Plant name (English)',
    hover_data={
        'Country/Area': True,
        'Owner': True,
        'Nominal crude steel capacity (ttpa)': ':.0f',
        'latitude': False,
        'longitude': False
    },
    size='Nominal crude steel capacity (ttpa)',
    color='Region',
    title='Geographic Distribution of Operating Steel Plants Worldwide (with Capacity Data)',
    size_max=15,
    projection='natural earth'
)

fig_geo.update_layout(
    height=600,
    geo=dict(
        showland=True,
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(204, 204, 204)',
        showocean=True,
        oceancolor='rgb(230, 245, 255)',
        showcountries=True,
        countrycolor='rgb(204, 204, 204)'
    )
)
fig_geo.show()

print(f"\nGeographic Coverage:")
print(f"  Countries represented: {df['Country/Area'].nunique()}")
print(f"  Regions represented: {df['Region'].nunique()}")
print(f"\nTop 10 countries by number of operating plants:")
print(df['Country/Area'].value_counts().head(10))

Latitude range:
  Min: -37.8314°
  Max: 66.3115°
  Mean: 30.5709°

Longitude range:
  Min: -123.1636°
  Max: 174.7281°
  Mean: 59.6227°

Plants with capacity data for visualization: 815 out of 868



Geographic Coverage:
  Countries represented: 80
  Regions represented: 7

Top 10 countries by number of operating plants:
Country/Area
China            289
India             74
United States     70
Japan             38
Iran              31
Russia            27
Türkiye           25
Brazil            22
Italy             22
Vietnam           20
Name: count, dtype: int64


In [ ]:
# Convert Plant age to numeric
df['Plant age (years)'] = pd.to_numeric(df['Plant age (years)'], errors='coerce')

print(f"\nPlants with age data: {df['Plant age (years)'].notna().sum()} out of {len(df)}")
print(f"\nAge Statistics:")
print(f"  Mean: {df['Plant age (years)'].mean():.2f} years")
print(f"  Median: {df['Plant age (years)'].median():.2f} years")
print(f"  Min: {df['Plant age (years)'].min():.0f} years")
print(f"  Max: {df['Plant age (years)'].max():.0f} years")
print(f"  Std Dev: {df['Plant age (years)'].std():.2f} years")

# Age distribution by bins
print(f"\nAge Distribution by Category:")
age_bins = [0, 10, 20, 30, 40, 50, 100, 300]
age_labels = ['0-10 years', '11-20 years', '21-30 years', '31-40 years', '41-50 years', '51-100 years', '100+ years']
df['age_category'] = pd.cut(df['Plant age (years)'], bins=age_bins, labels=age_labels, right=True)
print(df['age_category'].value_counts().sort_index())

# ============================================================================
# PLOTLY VISUALIZATIONS FOR PLANT AGE DISTRIBUTION
# ============================================================================

# 1. Histogram of plant ages
fig_age_hist = px.histogram(
    df.dropna(subset=['Plant age (years)']),
    x='Plant age (years)',
    nbins=40,
    title='Distribution of Plant Ages (Operating Plants)',
    labels={'Plant age (years)': 'Plant Age (years)'},
    color_discrete_sequence=['#2ca02c']
)
fig_age_hist.update_layout(
    xaxis_title='Plant Age (years)',
    yaxis_title='Number of Plants',
    showlegend=False,
    height=400
)
fig_age_hist.show()

# 2. Box plot of plant ages
fig_age_box = px.box(
    df.dropna(subset=['Plant age (years)']),
    y='Plant age (years)',
    title='Plant Age Distribution Statistics',
    points='all',
    color_discrete_sequence=['#2ca02c']
)
fig_age_box.update_layout(
    yaxis_title='Plant Age (years)',
    showlegend=False,
    height=400
)
fig_age_box.show()

# 3. Bar chart of age categories
age_category_counts = df['age_category'].value_counts().sort_index()
fig_age_cat = px.bar(
    x=age_category_counts.index.astype(str),
    y=age_category_counts.values,
    title='Number of Operating Plants by Age Category',
    labels={'x': 'Age Category', 'y': 'Number of Plants'},
    color=age_category_counts.values,
    color_continuous_scale='Greens'
)
fig_age_cat.update_layout(
    xaxis_title='Age Category',
    yaxis_title='Number of Plants',
    showlegend=False,
    height=400
)
fig_age_cat.show()

# 4. Scatter plot: Age vs Capacity
fig_age_capacity = px.scatter(
    df.dropna(subset=['Plant age (years)', 'Nominal crude steel capacity (ttpa)']),
    x='Plant age (years)',
    y='Nominal crude steel capacity (ttpa)',
    color='Region',
    size='Nominal crude steel capacity (ttpa)',
    hover_name='Plant name (English)',
    hover_data=['Country/Area', 'Owner'],
    title='Plant Age vs Capacity (Operating Plants)',
    size_max=15,
    opacity=0.7
)
fig_age_capacity.update_layout(
    xaxis_title='Plant Age (years)',
    yaxis_title='Crude Steel Capacity (ttpa)',
    height=500
)
fig_age_capacity.show()

print("\n" + "="*80)
print("KEY INSIGHTS FROM AGE ANALYSIS")
print("="*80)
print(f"  • Oldest operating plant: {df['Plant age (years)'].max():.0f} years old")
print(f"  • Newest operating plant: {df['Plant age (years)'].min():.0f} years old")
print(f"  • Median age: {df['Plant age (years)'].median():.1f} years")
print(f"  • Most common age range: {age_category_counts.idxmax()}")
print(f"  • Plants over 100 years old: {(df['Plant age (years)'] > 100).sum()}")
print("="*80)

### Question 3: Geographic Distribution
**Task:** Analyze the geographic distribution of steel plants.
- Which countries/regions have the most steel plants?
- What is the distribution of plants by company?


In [ ]:
# Count plants by country/region



In [ ]:
# Count plants by company



### Question 4: Capacity Analysis
**Task:** Analyze the capacity distribution.
- What is the total global steel production capacity?
- Which companies have the highest total capacity?
- How does capacity vary by region?


In [ ]:
# Calculate total capacity



In [ ]:
# Group by company and sum capacity



---
## Part 3: Geospatial Visualization with Plotly (15 minutes)

Create interactive maps to visualize the steel plants' locations and characteristics.


### Exercise 1: Basic Scatter Map
**Task:** Create a scatter map showing all steel plant locations.
- Use latitude and longitude for positioning
- Color points by country or region
- Add hover information showing plant name, company, and capacity


In [ ]:
# Create a scatter_geo or scatter_mapbox plot
# Hint: Use plotly.express.scatter_geo() or scatter_mapbox()



### Exercise 2: Sized Markers by Capacity
**Task:** Create a map where marker size represents plant capacity.
- Larger markers for higher capacity plants
- Color by company
- Include interactive hover details


In [ ]:
# Create scatter map with size parameter based on capacity



### Exercise 3: Density Heatmap
**Task:** Create a density map showing concentration of steel plants.
- Use Plotly's density_mapbox to show clustering
- Identify regions with high plant density


In [ ]:
# Create density heatmap
# Hint: Use plotly.express.density_mapbox()



---
## Part 4: Merging Environmental Data with Assets

Integrate environmental data (e.g., air quality, emissions, proximity to water sources) with steel plant locations.


### Exercise 1: Load Environmental Data
**Task:** Load the environmental dataset and inspect it.

- [Litpop database](https://www.research-collection.ethz.ch/entities/researchdata/12dcfc4f-9d03-463a-8d6b-76c0dc73cdc8)

- Expected columns: location_id, latitude, longitude, population density, activity etc.


In [ ]:
# Load environmental data



In [ ]:
# Inspect environmental data



### Exercise 2: Spatial Join or Nearest Neighbor Matching
**Task:** Merge environmental data with steel plants based on geographic proximity.
- Use nearest neighbor matching or spatial join
- Consider using geopandas for distance calculations
- Match each plant to the nearest environmental monitoring station


In [ ]:
# Calculate distances or perform spatial join
# Hint: You might calculate haversine distance or use a spatial library



In [ ]:
# Merge datasets



### Exercise 3: Visualize Merged Data
**Task:** Create a map showing steel plants colored by environmental metrics.
- Color plants by air quality index or other environmental indicators
- Size by capacity
- Add hover details with both plant and environmental information


In [ ]:
# Create visualization of merged data



---
## Part 5: Company-Level Aggregation

Aggregate data at the company level to analyze corporate footprints.


### Exercise 1: Aggregate Metrics by Company
**Task:** Group plants by company and calculate aggregate metrics.
- Total capacity per company
- Number of plants per company
- Average environmental metrics per company
- Geographic spread (e.g., number of countries)


In [ ]:
# Group by company and aggregate



### Exercise 2: Company Headquarters or Centroid
**Task:** Calculate a representative location for each company.
- Option 1: Use the centroid of all plant locations
- Option 2: Use the location of the largest plant
- Option 3: Assign actual headquarters coordinates


In [ ]:
# Calculate company representative locations



### Exercise 3: Visualize Company-Level Data
**Task:** Create a map showing companies with aggregated metrics.
- Show one marker per company at the representative location
- Size by total capacity
- Color by average environmental impact
- Hover information with company summary statistics


In [ ]:
# Create company-level visualization



---
## Part 6: Streamlit Dashboard Integration

Prepare your visualizations for deployment in a Streamlit dashboard.


### Exercise 1: Create Dashboard Script Structure
**Task:** Create a Streamlit app file (`dashboard.py`) with the following structure:

```python
# Import streamlit and other necessary libraries

# Set page configuration

# Title and description

# Sidebar for filters
# - Company selector
# - Region/country filter
# - Capacity range slider

# Main content area
# - KPI metrics (total plants, total capacity, etc.)
# - Interactive map
# - Data table

# Footer with data sources and notes
```


### Exercise 1: Prepare Data for Dashboard
**Task:** Save your processed data to files that the dashboard will load.
- Export cleaned plant data
- Export merged environmental data
- Export company-level aggregations
- Save as CSV or Parquet for efficient loading


In [ ]:
# Save processed datasets



### Exercise 2: Display relevant information from your exploratory analysis into the dashboard

In [ ]:
# This cell is for notes/observations about your dashboard
# What works well?
# What could be improved?
# Any performance issues with large datasets?



---
## Lab Summary and Key Takeaways

**What you learned:**
- How to perform EDA on geospatial datasets
- Creating interactive maps with Plotly for geospatial data
- Merging spatial datasets based on geographic proximity
- Aggregating geospatial data at different levels (asset vs. company)
- Building interactive dashboards with Streamlit

**Next Steps:**
- Explore other geospatial libraries (GeoPandas, Folium, Kepler.gl)
- Learn about coordinate reference systems (CRS) and projections
- Practice with other datasets (buildings, utilities, transportation)
- Deploy your dashboard to Streamlit Cloud or other hosting services
